In [1]:
import pickle as pickle 
import random
import operator
#from itertools import permutations 
from itertools import combinations 
import timeit

In [4]:
import numpy as np
import pandas as pd

In [5]:
from torch import softmax
from transformers import BertForNextSentencePrediction, BertTokenizer

In [7]:
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

### Bert NextSentencePrediction

In [60]:
def bert_nsp(seq_A, seq_B):
    encoded = tokenizer.encode_plus(seq_A, text_pair=seq_B, return_tensors='pt',max_length=10000)
    try: 
        seq_relationship_logits = model(**encoded)[0]
        probs = softmax(seq_relationship_logits, dim=1)
        similarity = probs.detach().numpy()[0][0]
    except:
        similarity = 0
    return similarity

### Functions for generating Bert Similarity Tables

In [61]:
def One_prob_clustering(posts, m,n,t):
    unselected_posts = posts.copy()
    clusters = {}
    while len(unselected_posts)> 0:
        select_post = random.choice(unselected_posts)
        sim_dict = {}
        for p in unselected_posts:
            similarity = bert_nsp(t[select_post][0], t[p][1])
            sim_dict[p] = similarity
        sorted_sim_list = [key for key,value in sorted(sim_dict.items(), key=operator.itemgetter(1),reverse=True)]
        try:
            most_similar = sorted_sim_list[0:int(np.ceil(n/m))]
        except:
            most_similar = sorted_sim_list[0:end]
        clusters[select_post] = most_similar
        for post in most_similar:
            unselected_posts.remove(post)
    return clusters

In [62]:
def Merge_multiple_prob_clustering(p, m, posts,n,t):
    similarity_table = {}
    for i in posts:
        similarity_table[i] = np.zeros(n)
    for i in range(p):
        one_probabilistic_clustering = One_prob_clustering(posts, m, n,t)
        for j in one_probabilistic_clustering.keys():
            one_cluster =  [j] + one_probabilistic_clustering[j]
            all_similar_pairs = combinations(one_cluster, 2)
            for k in all_similar_pairs:
                row_index = posts.index(k[1])
                similarity_table[k[0]][row_index] += 1
    return similarity_table

### n = 1000 sample 

In [70]:
with open('data/bert_title_text.pickle', 'rb') as handle:
    d = pickle.load(handle)

In [71]:
posts = list(d.keys())
n = len(posts)
p =5 
m =30 #hyperparameter from the paper
t = d.copy()

In [ ]:
start = timeit.default_timer()
similarity_table = Merge_multiple_prob_clustering(p, m, posts,n,t)
stop = timeit.default_timer()

In [ ]:
with open('bert_similarity_table_1000.pickle', 'wb') as handle:
    pickle.dump(similarity_table, handle)

In [ ]:
print('Time: ', stop - start) 

### n = [2200, 4400, 8800, 13200]

In [63]:
for s in [2200,4400,8800,13200]:
    input_file = 'data/bert_title_text%d.pickle' %s
    with open(input_file, 'rb') as handle:
        d = pickle.load(handle)
    posts = list(d.keys())
    n = len(posts)
    p =5 
    m =3
    t = d.copy()

    start = timeit.default_timer()
    similarity_table = Merge_multiple_prob_clustering(p, m, posts,n,t)
    stop = timeit.default_timer()
    ramdom 
    output_filename = 'bert_similarity_table_%d.pickle' %s
    with open(output_filename, 'wb') as handle:
        pickle.dump(similarity_table, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print('Time for %d: ' %s, stop - start) 

Time for 2200:  5278.650366899998
Time for 4400:  10481.6847509


KeyboardInterrupt: 